# CLS Vector Analysis for Original IMDB Dataset 

## Imports & Inits

In [ ]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

import pdb, pickle, sys, warnings, itertools, re
warnings.filterwarnings(action='ignore')
sys.path.insert(0, '../scripts')

from IPython.display import display, HTML

import pandas as pd
import numpy as np
from argparse import Namespace
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

np.set_printoptions(precision=4)
sns.set_style("darkgrid")
%matplotlib inline

In [ ]:
from datasets import load_dataset
imdb = load_dataset('imdb')

In [ ]:
imdb.set_format(type='pandas')
df = imdb['train'][:]
df.head()

In [ ]:
def label_int2str(row):
  return imdb['train'].features['label'].int2str(row)

df['label_name'] = df['label'].apply(label_int2str)
df.head()

In [ ]:
df['label_name'].value_counts(ascending=True).plot.barh()